In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import pandas as pd
import awswrangler as wr
from matplotlib import pyplot as plt
import seaborn as sns

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url

In [3]:
audience = ["68e926d4b236f67910699632","68e92627b236f67910678fbd","68e92747b236f679106a375c"]
customer = "Rosental"
date = "20251019"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [9]:
result = pd.DataFrame()
for aud in audience:
    print(aud)
    df = wr.s3.read_parquet(f"s3://{account_id}/targeting.history/{date}/{aud}.parquet")
    df["load_audience"] = aud
    print(len(df), df["treatment"].value_counts())
    result = pd.concat([result, df])
result.head()

In [10]:
targeted = result[result["treatment"]!="control"]
res_anonymousids = pd.DataFrame(columns=["anonymousId"])
for aud in audience:
    temp = targeted[targeted["load_audience"] == aud]
    temp = temp[["load_audience","anonymousId"]].rename(columns={"load_audience":aud})
    res_anonymousids = pd.merge(res_anonymousids, temp, how="outer", on="anonymousId")
res_anonymousids

In [18]:
res_dict = {}
for aud1 in audience:
    for aud2 in audience:
        if aud1 == aud2:
            continue
        print(aud1, aud2)
        aud1_entries = res_anonymousids[aud1].count()
        aud2_entries = res_anonymousids[aud2].count()
        aud1_and_aud2 = res_anonymousids[(res_anonymousids[aud1].notna()) & (res_anonymousids[aud2].notna())]
        aud1_and_aud2 = aud1_and_aud2[aud1].count()
        res_dict[(aud1, aud2)] = {aud1: aud1_entries, aud2: aud2_entries, "schnittmenge": aud1_and_aud2}
res_dict